In [ ]:
from Other_To_nparray import generate_data, traintest_split
from keras.models import load_model
from model_evalution import evalution_metrics
from Model.PC_6_model import t_m
import os

In [ ]:
# path
data_output = 'Output/PC_6_all/'
model_output = 'Output/PC_6_all/Model/'
model_name='PC_6'
if not os.path.isdir(data_output):
    os.mkdir(data_output)

In [ ]:
train_data, train_labels = generate_data(pos_fasta='4DB_independent_6623.fasta', neg_fasta='Train_Negative_6623.fasta', method='PC_6')

In [ ]:
model = load_model('Output/PC_6_ver2/Model/PC_6_best_weights.h5')

In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D
from keras.layers import LSTM,Add,Input
from keras.layers import LeakyReLU
from keras import models
from keras import optimizers
import os
os.environ['CUDA_VISIBLE_DEVICES']='3'

model.compile(optimizer=optimizers.Adam(lr=1e-4, clipnorm=1),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                              patience=50,verbose=1)
best_weights_filepath = model_output+'/%s_best_weights.h5'%model_name
saveBestModel = keras.callbacks.ModelCheckpoint(best_weights_filepath, 
                                                    monitor='val_loss', 
                                                    verbose=1, 
                                                    save_best_only=True, 
                                                    mode='auto')
final_filepath = model_output+'/%s_final.h5'%model_name
saveFinalModel = keras.callbacks.ModelCheckpoint(final_filepath, 
                                                    save_best_only=False)
e_s = keras.callbacks.EarlyStopping(monitor='val_loss',
                                          min_delta=0,
                                          patience=300,
                                          verbose=0, mode='max')
CSVLogger = keras.callbacks.CSVLogger(model_output+"/%s_csvLogger.csv"%model_name,separator=',', append=False)

t_m=model.fit(train_data,train_labels,shuffle=True,validation_split=0.1, 
                epochs=250, batch_size=int(0.5*len(train_data)),callbacks=[saveBestModel,CSVLogger,e_s,saveFinalModel,reduce_lr])